# Implementazione di un modello semplificato di Expected Goals
In questa lezione useremo il dataset con tutti i tiri della Serie A 2022/23 per creare un modello semplificato di Expected Goals. Questi modelli sono solitamente creati su dataset molto più ampi, con tutti i tiri di più stagioni e da più campionati. In questo caso faremo un esempio semplificato con un dataset ridotto con lo scopo di introdurre il metodo, più che di raggiungere un risultato finale.


# Esplorazione del dataset
Iniziamo caricando le librerie necessarie e leggendo il nostro dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, log_loss, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.ndimage import gaussian_filter
from mplsoccer.pitch import Pitch, VerticalPitch
from database.read_db import *

def eval_q(x):
    return eval(x.replace("null", "None"))

def get_numeric_q(x,qid):
    for l in x:
        if l['@attributes']['qualifier_id'] == str(qid):
            return float(l['@attributes']['value'])

def get_binary_q(x,qid):
    bl = []
    for l in x:
        if l['@attributes']['qualifier_id'] == str(qid):
            bl.append(True)
        else:
            bl.append(False)
    bl = np.asarray(bl)
    if bl.any():
        return True
    else:
        return False

In [ ]:
df = read_db('ds.team_player_events')
dfs = df[df.type_id.isin([13, 14, 15, 16]) & (df.own_goal == False)]

Vediamo come sono fatti i nostri dati.

In [ ]:
dfs.info()

Abbiamo a disposizione circa 10000 tiri con informazioni posizionali e contestuali. Dato che l'obbiettivo è di modellare la probabilità di gol per i nostri tiri, creiamo una colonna binaria corrispondente che sarà il target del modello.

In [ ]:
dfs['goal'] = (dfs.type_id == 16).astype(int)
dfs.goal.value_counts(normalize = True)

Individuata la nostra variabile target, passiamo ad occuparci delle variabili predittive, o features, basate sulla posizione. Abbiamo a disposizione nel nostro dataframe le variabili `x` e `y` che rappresentano le coordinate di ogni tiro secondo la convenzione di Opta (valori fra 0 e 100, direzione di attacco da sinistra a destra). Usiamole per andare a vedere la distribuzione dei nostri tiri.

In [ ]:
pitch = VerticalPitch(pitch_type = 'opta', line_zorder = 2, half = True)

f, ax = pitch.draw(figsize=(5,5))
bin_statistic = pitch.bin_statistic(dfs.x, dfs.y, statistic='count', bins=(30,30))
pcm = pitch.heatmap(bin_statistic, cmap='Blues', ax=ax)
cbar = plt.colorbar(pcm, shrink = 0.5, pad = 0.01)
plt.show()

Vediamo che c'è una forte concentrazione di tiri sul dischetto del rigore. Come detto nelle prime lezioni, il modello di xG vero e proprio viene costruito escludendo i rigori, che hanno un valore di xG fissato a quello del conversion rate. Filtriamo quindi i nostri dati escludendo i rigori.

In [ ]:
df2 = dfs[dfs.penalty == False]

In [ ]:
pitch = VerticalPitch(pitch_type = 'opta', line_zorder = 2, half = True)

f, ax = pitch.draw(figsize=(5,5))
bin_statistic = pitch.bin_statistic(df2.x, df2.y, statistic='count', bins=(30,30))
pcm = pitch.heatmap(bin_statistic, cmap='Blues', ax = ax)
cbar = plt.colorbar(pcm, shrink = 0.5, pad = 0.01)
plt.show()

Vediamo che la maggior parte dei tiri avvengono dentro l'area. E se volessimo calcolare il conversion rate in questa griglia? Per un modello di xG basato solamente sulla posizione, questo valore dovrebbe essere quello che si cerca di modellare. Possiamo visualizzarlo aggiungendo la colonna binaria `goal` come terzo parametro nella funzione `bin_statistic`, e cambiando la funzione di aggregazione da `count` a `mean`.

In [ ]:
pitch = VerticalPitch(pitch_type = 'opta', line_zorder = 2, half = True)

f, ax = pitch.draw(figsize=(5,5))
bin_statistic = pitch.bin_statistic(df2.x, df2.y, df2.goal, statistic='mean', bins=(30,30))
pcm = pitch.heatmap(bin_statistic, cmap='Blues', ax=ax)
cbar = plt.colorbar(pcm, shrink = 0.5, pad = 0.01)
plt.show()

Vediamo che in generale il conversion rate scende all'aumentare della distanza dalla porta, e per coordinate molto defilate rispetto allo specchio. Vediamo anche che ci sono dei bin con valori estremi del conversion rate. Questo è dovuto alla poca numerosità dei tiri in queste zone, come possiamo vedere stampando su ogni bin il numero di tiri corrispondente.

In [ ]:
pitch = VerticalPitch(pitch_type = 'opta', line_zorder = 2, half = True)

f, ax = pitch.draw(figsize=(10,10))
bs_conv = pitch.bin_statistic(df2.x, df2.y, df2.goal, statistic='mean', bins=(30,30))
bs_conv['statistic'] = np.nan_to_num(bs_conv['statistic'])
bs_count = pitch.bin_statistic(df2.x, df2.y, statistic='count', bins=(30,30))
pcm = pitch.heatmap(bs_conv, cmap='Blues', ax=ax)
cbar = plt.colorbar(pcm, shrink = 0.5, pad = 0.01)
labels = pitch.label_heatmap(bs_count, color='black',fontsize = 8,
                             ax=ax, ha='center', va='center',
                             str_format='{:.0f}', exclude_zeros = True)
plt.show()

Impostando un modello statistico, l'obbiettivo sarà di ricostruire una dipendenza corretta della nostra variabile target dalle coordinate, in modo da evitare di predire valori estremi di questo tipo.

#Preparazione delle variabili predittive
Abbiamo visto che il conversion rate sembra dipendere dalla distanza dalla porta e da quanto i tiro è defilato rispetto allo specchio. Questo ci suggerisce di trasformare le nostre coordinate da `x` e `y` in un sistema radiale, dove ogni punto è identificato dalla sua distanza dal centro della porta e dall'angolo di porta.

La distanza dal centro della porta si calcola tramite considerazioni geometriche molto semplici. In primo luogo creiamo due nuove variabili con la distanza dalla porta in direzione verticale e laterale. Nel caso della `x` si tratta solo di invertire le coordinate, mentre per la `y` posizioniamo lo zero della nuova coordinata su 50.

In [ ]:
df2['xGoal'] = 100 - df2.x
df2['yGoal'] = df2.y - 50.

La distanza dal centro della porta sarà quindi semplicemente la norma del vettore identificato da queste coordinate, ossia (in termini più semplici) l'ipotenusa del triangolo formato dalle due coordinate:

In [ ]:
df2['distance'] = np.sqrt(df2.xGoal**2 + df2.yGoal**2)

Per quanto riguarda l'angolo di porta, si tratta dell'angolo sottinteso dall'ampiezza della porta per ogni punto, che aumenta quanto più il tiro si trova direttamente di fronte alla porta e diminuisce quanto più il tiro è defilato.

In [ ]:
pitch = VerticalPitch(pitch_type = 'opta', line_zorder = 1, half = True)

f, ax = pitch.draw(figsize=(5,5))
pitch.lines(
    [80, 80],
    [10, 10],
    [100, 100],
    [50-9.6/2., 50+9.6/2.],
    lw = 2,
    color = 'blue',
    ax = ax
)

pitch.lines(
    [85, 85],
    [55, 55],
    [100, 100],
    [50-9.6/2., 50+9.6/2.],
    lw = 2,
    color = 'red',
    ax = ax
)
plt.show()

L'angolo di porta si calcola tramite considerazioni trigonometriche su cui non entreremo in dettaglio, nota la larghezza della porta, che è pari a 7.32 metri, corrispondenti a 9.6 in unità Opta.

In [ ]:
gsize = 9.6
df2['angle'] = np.arctan(gsize *df2.xGoal /(df2.xGoal**2 + df2.yGoal**2 - (gsize/2)**2))
df2.loc[(df2.angle < 0.), 'angle'] = np.pi + df2.angle

Vediamo come visualizzare la dipendenza del conversion rate da queste variabili, calcolandolo per valori raggruppati di distanza e angolo tramite la funzione `qcut` di Pandas.

In [ ]:
df2['dist_bins'] = pd.qcut(df2['distance'],q=50)
df2.dist_bins

In [ ]:
dist_prob = df2.groupby('dist_bins')['goal'].mean()
dist_mean = df2.groupby('dist_bins')['distance'].mean()

f, ax = plt.subplots(figsize = (5,5))
ax.scatter(x = dist_mean, y = dist_prob)
ax.set_xlabel('Distance')
ax.set_ylabel('Conversion rate')
plt.show()

In [ ]:
df2['angle_bins'] = pd.qcut(df2['angle'],q=50)

angle_prob = df2.groupby('angle_bins')['goal'].mean()
angle_mean = df2.groupby('angle_bins')['angle'].mean()

f, ax = plt.subplots(figsize = (5,5))
ax.scatter(x = angle_mean, y = angle_prob)
ax.set_xlabel('Goal angle')
ax.set_ylabel('Conversion rate')
plt.show()

Troviamo quindi conferma che il conversion rate aumenta con il diminuire della distanza e con l'aumentare dell'angolo di porta, come atteso.

# Implementazione del modello
Ora che abbiamo le nostre variabili predittive e la nostra variabile target, procediamo con l'implementazione del modello come abbiamo visto nella lezione precedente: definiamo la matrice `X` per le features e `y` per la variabile target, creiamo il nostro regressore usando `LogisticRegression`, alleniamolo tramite `fit`, e calcoliamo le probabilità, ossia i valori di xG, con `predict_proba`.

In [ ]:
X = df2[['distance', 'angle']].values
y = df2.goal.values

clf = LogisticRegression()
clf.fit(X,y)

df2['xG'] = clf.predict_proba(X)[:,1]

Andiamo per prima cosa a visualizzare i risultati del nostro modello sul campo con una heatmap analoga a quella usata in precedenza per il conversion rate.

In [ ]:
pitch = VerticalPitch(pitch_type = 'opta', line_zorder = 2, half = True)

f, ax = pitch.draw(figsize=(5,5))
bin_statistic = pitch.bin_statistic(df2.x, df2.y, df2.xG, statistic='mean', bins=(30,30))
pcm = pitch.heatmap(bin_statistic, cmap='Blues', ax=ax)
cbar = plt.colorbar(pcm, shrink = 0.5, pad = 0.01)
plt.show()

Vediamo che i valori sembrano abbastanza sensati. Per valutare in modo quantitativo il nostro modello possiamo usare il ROC AUC score, come visto nella lezione precedente.

In [ ]:
roc_auc = roc_auc_score(y,df2.xG.values)

fpr, tpr, thresholds = roc_curve(y, df2.xG.values)

f, ax = plt.subplots(figsize = (5,5))
ax.plot(fpr, tpr, lw = 2, color = 'black')
ax.plot(fpr, fpr, color = 'black', ls = '--')
xarr = np.linspace(0, 1, len(fpr))
ax.fill_between(x = fpr, y1 = fpr, y2 = tpr, alpha = 0.5)
ax.annotate(f'ROC AUC score: {roc_auc:.2f}', xy = (0.1, 0.9), xycoords = 'axes fraction', fontsize = 14)
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
plt.show()

Per valutare l'accuratezza del modello in termini di probabilità, e quindi la precisione dei singoli valori di xG, possiamo valutare anche la LogLoss, vista sempre nella lezione precedente.

In [ ]:
ll = log_loss(y,df2.xG.values)
print(ll)

Come detto, la LogLoss va confrontata col modello "ingenuo", che in questo caso equivale ad assegnare a tutti i tiri un valore di xG pari al conversion rate medio.

In [ ]:
#calcolo valore medio variabile target
avg = y.mean()

#creazione array NumPy con stesso valore per ogni elemento dei dati
avg_arr = np.full(len(df2), avg)

#calcolo LogLoss naive
naive_ll = log_loss(y, avg_arr)

print(f'LogLoss naive: {naive_ll:.2f}, LogLoss modello: {ll:.2f}')

Vediamo che il nostro semplice modello di xG risulta già più esplicativo rispetto al conversion rate.

Lo score è abbastanza buono per un modello così semplice. Come ulteriore verifica calcistica possiamo aggregare i dati per giocatore tramite `groupby` e confrontare gol e xG.

In [ ]:
dfg = df2.groupby('player_id')[['goal', 'xG']].sum()

f, ax = plt.subplots(figsize = (5,5))
sns.regplot(data = dfg, x = 'xG', y = 'goal', ax = ax)
plt.show()

Sembra esserci una correlazione abbastanza buona, che possiamo quantificare anche calcolando la matrice di correlazione con $R^2$ usando `corr`.

In [ ]:
dfg.corr()**2

# Bonus: estrazione informazioni su tipo di assist

Andiamo a vedere come legare l'informazione sul tipo di assist ai nostri dati sui tiri. Questa informazione è contenuta nel qualifier 55 "Related event ID", che indica l'ID numerico dell'evento passaggio nei casi in cui il tiro sia stato preceduto da un passaggio. Questo tipo di ID `event_id` è un contatore per ognuna delle due squadre all'interno della partita, per cui possiaom ricavarlo facendo un merge su `event_id`, `team_id` e `game_id`.

In [ ]:
df2['Q'] = df2.Q.apply(eval_q)

In [ ]:
df2['related_event_id'] = df2.Q.apply(get_numeric_q, args = ([55]))

In [ ]:
df = df[df.type_id == 1].dropna(subset = ['Q'])
df['Q'] = df.Q.apply(eval_q)
df['through_ball'] = df.Q.apply(get_binary_q, args = ([4]))

In [ ]:
df2 = df2.merge(df[['event_id', 'team_id', 'game_id', 'through_ball', 'long_ball', 'cutback', 'cross']], left_on = ['related_event_id', 'team_id', 'game_id'], right_on = ['event_id', 'team_id', 'game_id'], how = 'left', suffixes=['', '_prev'])

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'through_ball' : 'through_ball_prev'}, inplace=True)